Click the button below if you want to see the code behind the widgets. If you do click  and want to go back to the 'no-code' view, scroll down until you find the 'hide code' button.

In [1]:
"""
MIT License

Copyright (c) 2020 Sylvain Barde - University of Kent

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
"""

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams
from IPython.display import display, HTML

javascript_functions = {False: "hide()", True: "show()"}
button_descriptions  = {False: "Click to show code", True: "Click to hide code"}

def toggle_code(state):
    """
    Toggles the JavaScript show()/hide() function on the div.input element.
    """

    output_string = "<script>$(\"div.input\").{}</script>"
    output_args   = (javascript_functions[state],)
    output        = output_string.format(*output_args)

    display(HTML(output))
    
def button_action(value):
    """
    Calls the toggle_code function and updates the button description.
    """
    state = value.new

    toggle_code(state)

    value.owner.description = button_descriptions[state]

def taylor_widget(xMin_init = -6, xMax_init = 6, yMin_init = -1.5, yMax_init = 1.5, 
                  func_init = 'Sine', numTerms_init = 6):

    # Declare widgets for interactive input
    xMin_slider = widgets.FloatSlider(min=-1000,
                                 max=1000,
                                 description=r'Min $x$:',
                                 value = xMin_init,
                                 continuous_update =False)
    xMax_slider = widgets.FloatSlider(min=-1000,
                                 max=1000,
                                 description=r'Max $x$:',
                                 value = xMax_init,
                                 continuous_update =False)
    yMin_slider = widgets.FloatSlider(min=-1000,
                                 max=1000,
                                 description=r'Min $y$:',
                                 value = yMin_init,
                                 continuous_update =False)
    yMax_slider = widgets.FloatSlider(min=-1000,
                                 max=1000,
                                 description=r'Max $y$:',
                                 value = yMax_init,
                                 continuous_update =False)
    func_list = widgets.Dropdown(options=['Exponential', 'Logarithm', 'Sine'],
                                value = func_init,
                                description=r'Pick function:',
                                disabled=False)
    numTerms_slider = widgets.IntSlider(min=1,
                                 max=24,
                                 description=r'N° of terms:',
                                 value = numTerms_init,
                                 continuous_update =False)
    
    # Link widgets as required
    widgets.jslink((xMin_slider,'value'),(xMax_slider,'min'))
    widgets.jslink((xMax_slider,'value'),(xMin_slider,'max'))
    widgets.jslink((yMin_slider,'value'),(yMax_slider,'min'))
    widgets.jslink((yMax_slider,'value'),(yMin_slider,'max'))

    def taylor_plot(xMin, xMax, yMin, yMax, func, numTerms):

        # Prepare dicts of log functions and color codes
        labels = {'Exponential':'e^x',
                  'Logarithm':'ln(x)',
                  'Sine':'sin(x)'}
        
        funcDict = {'Exponential': lambda x: np.exp(x),
                   'Logarithm': lambda x: np.log(x.clip(1e-6)),
                   'Sine': lambda x: np.sin(x)}

        taylorDict = {'Exponential': lambda x,n: (x**n)/np.math.factorial(n),
                   'Logarithm': lambda x,n: ((-1)**(n+1))*((x-1)**n)/n if n > 0 else 0,
                   'Sine': lambda x,n: ((-1)**n)*(x**(2*n+1))/np.math.factorial(2*n+1)}

        colors = ['b','r','g','m','c','y',
                  'b--','r--','g--','m--','c--','y--',
                  'b:','r:','g:','m:','c:','y:',
                  'b-.','r-.','g-.','m-.','c-.','y-.']

        # Create an 'x' vector
        x = np.arange(xMin,xMax,(xMax-xMin)/500)

        # Create figure, calculate and plot Taylor function for each term added
        fig, ax = plt.subplots(figsize=(20,10))

        yTaylor = np.zeros(len(x))
        for n in range(numTerms):         
            taylorTerm = np.asarray(
                            np.clip(taylorDict[func](x,n), -1000, 1000), 
                            dtype = np.double)
            yTaylor += taylorTerm
            ax.plot(x, yTaylor,colors[n], linewidth=2, alpha=0.6, 
                    label='Term {:d}'.format(n))

        y = funcDict[func](x)
        ax.plot(x, y,'k', linewidth=2, label=r'$\quad y = {:s}$'.format(labels[func]))

        # Add legend and format axes to look nice
        ax.legend(loc='lower center', frameon=False,prop={'size':20},ncol=6,
                   bbox_to_anchor=(0.5, -0.25))

        ax.autoscale(enable=True, axis='both', tight=True)
        ax.set_ylim(top = yMax, bottom = yMin)
        ax.set_xlim(right = xMax, left = xMin)
        
        if yMin > 0:
            ax.spines['bottom'].set_position(('data',yMin))
        elif yMax < 0:
            ax.spines['bottom'].set_position(('data',yMax))
        else:       
            ax.spines['bottom'].set_position('zero')
        ax.spines['top'].set_visible(False)
        if xMin > 0:
            ax.spines['left'].set_position(('data',xMin))
        elif xMax < 0:
            ax.spines['left'].set_position(('data',xMax))
        else:
            ax.spines['left'].set_position('zero')
        ax.spines['right'].set_visible(False)
        ax.set_xlabel(r'$x$', fontdict = {'fontsize': 25},position=(1, 0))
        ax.set_ylabel(r'$y$', fontdict = {'fontsize': 25},position=(0, 1), rotation=0)
        ax.plot(1, 0, ">k", transform=ax.get_yaxis_transform(), clip_on=False)
        ax.plot(0, 1, "^k", transform=ax.get_xaxis_transform(), clip_on=False)
        plt.tick_params(labelsize=20)
        plt.tight_layout()
    
    out = widgets.interactive_output(taylor_plot, {'xMin': xMin_slider,
                                                'xMax': xMax_slider,
                                                'yMin': yMin_slider,
                                                'yMax': yMax_slider,
                                                'func': func_list, 
                                                'numTerms': numTerms_slider})

    output = widgets.VBox([out,
                  widgets.HBox([xMin_slider,
                               xMax_slider,
                               yMin_slider,
                               yMax_slider]),
                  widgets.HBox([func_list, 
                                numTerms_slider])])
    display(output)    
    

In [2]:
state = False
toggle_code(state)

button = widgets.ToggleButton(state, description = button_descriptions[state])
button.observe(button_action, "value")

display(button)

ToggleButton(value=False, description='Click to show code')

### Deep(er) dive: what's the point of polynomials?


If you think back to the maths you did during your school days, you probably remember working a lot with polynomials: finding their roots, finding their turning points (locations where they switch from being upwards sloping to downward sloping, or the other way round), etc. One thing you may well have been wondering at the time, and perhaps still to this day, was: what is the point of this?? Polynomials are an important part of mathematics, so much so that they form the bread and butter of what you do in secondary school, but my experience is that while you get taught **how** to work with them, often you don't get taught **why** they are such important functions, at least not at school. So the purpose of this deeper dive is to give you some insight into the importance of polynomials.

#### Wait, what's a polynomial?

Before we jump in, I might need to clarify my concepts so that everyone is on the same page. A polynomial is a function $y=f(x)$ of a single variable $x$ that has the following structure:

$$y = a + bx + cx^2 + dx^3 + ex^4 + ... $$

Where $a,b,c,d,e,\dots$ are parameters. So a polynomial is the sum of several terms, each of which contains $x$ raised to a different power. Each of these individual terms is called a ***mono*mial**, and the sum of all the terms is the ***poly*nomial**. This comes from the greek prefixes **mono-** and **poly-**, which respectively mean *single* and *many*.

Just to be clear, the notation used above is meant to show the general case. In practice a polynomial can have a finite number of terms, and can have any powers of $x$ you want. So this means that linear functions $y=ax +b$ are polynomials, quadratic equations $y = a^2 + bx + cx$ are also polynomials, but so is $y = 10 - 4x^2 + 6x^5$, or any such sum of powers of $x$.

#### Taylor polynomials as approximations of functions

Now that this is out of the way, we can look at what you can do with polynomials. The concept that I want to illustrate for you in a very visual way is the idea that polynomials can be used to provide approximations to functions. Don't worry about the functions that I am using as examples. They are just there to provide good illustrations and nice diagrams, and in any case, logarithms and exponentials are something we haven't yet looked at.

Instead, focus on the core idea: The three functions used here (the sine, the exponential and the logarithm) all have different shapes, yet in each case they can be approximated by a **suitable** polynomial. What we mean by suitable is that the parameters ($a$, $b$, $c$, $d$, etc. in the equation above) have to take specific values. The key trick which involves more advanced maths, of course, is how to pick those 'correct' values.

I'm not going to go into detail about the method you use for selecting the parameter values, as that would definitely be too deep a dive. Instead, I want to outline the key theoretical idea, which has to do with the concept of an *analytical* function. A function is said to be *analytical* when it can be written as a polynomial. Obviously this means that polynomial functions qualify! But more interestingly, many other functions, like the ones in our diagrams, also have this property.

I'm going to put below the polynomial that each of the three functions is equal to, but as I said don't worry about where the numbers come from. Instead, there are two things I want you to note.
1. A minor thing (for clarity): some of these expressions contain an exclamation mark, this is called the factorial function. It's actually very easy to understand, it's just a compact way of writing the product of successive integers. For instance $3! = 1\times 2 \times 3$, or as another example $5! = 1\times 2 \times 3 \times 4 \times 5$.
2. A major thing: In all cases, the polynomials are *infinite* in length, so they are actually a series which converges towards the value of the function in a given interval. The specific series linked to each function is called its Taylor series, named after English mathematician Brook Taylor, who came up with the systematic method for finding them in 1715. He was not the first to work with these ideas, mathematicians in 14th century India had already discovered much of these properties of polynomial series. Taylor did prove the theorem, however, so got the credit. (In another twist, the series below are also often called *Maclaurin* series, after the Scottish mathematician who worked on them as special cases of the more general Taylor theorem... Ideas in maths rarely have a clean attribution!)

For the sine function, the Taylor series is:

$$\sin(x) = x - \frac{x^3}{3!} + \frac{x^5}{5!} - \frac{x^7}{7!} + \frac{x^9}{9!} - \dots$$

For the exponential, it is:

$$e^x = 1 + x + \frac{x^2}{2!} + \frac{x^3}{3!} + \frac{x^4}{4!} + \frac{x^5}{5!} + \dots$$

and finally, for the natural logarithm we have:

$$\ln x = x - \frac{(x-1)^2}{2} + \frac{(x-1)^3}{3} - \frac{(x-1)^4}{4} + \frac{(x-1)^5}{5} - \dots$$

So the first important idea is that lots of important mathematical functions have this important property that can be written using an infinite series. In practice, obviously, you can't actually calculate these infinite sums! This is where the second idea comes in. If you stop the series after a certain number of terms, what you have is:
1. A finite polynomial that you can actually calculate and 
2. A function that *approximates* the initial function (around $x=0$ for the sine and exponential, around $x=1$ for the logarithm). So the full, infinite series is called the Taylor **series**, while the finite polynomial is called the Taylor **polynomial**. As you will see from the diagrams below, the more terms you include, the better the approximation.


### Taylor polynomials for the sine function

The first example, below, is what you get for the sine function. As you can see, the polynomial made up of the first 6 terms works pretty well between -4 to 4. You can include up to 24 terms if you want, as well as change the bounds of the diagram so feel free to play around and see how extra terms extend the range of this approximation.


In [7]:
taylor_widget()

### Taylor polynomials for the exponential function

Here is the same idea, but this time for the exponential function. I've put 12 terms initially, so that you can see how the approximation behaves differently for positive and negative $x$. For negative $x$, you get a very pretty herringbone pattern, where each successive term improves the approximation a bit then diverges alternatively up or down. You can add more terms if you want to see the pattern continue.

In [3]:
taylor_widget(xMin_init = -6, xMax_init = 6, yMin_init = -2, yMax_init = 6, func_init = 'Exponential', numTerms_init = 12)

The second plot focuses on the right hand side of the exponential, so that you can see that on this side of the function each extra term you add to the polynomial makes you converge closer to the true value of the function.

In [4]:
taylor_widget(xMin_init = 0, xMax_init = 2.5, yMin_init = 0, yMax_init = 12, func_init = 'Exponential', numTerms_init = 12)

### Taylor polynomials for the logarithm function

Moving on to the logarithm, we get something similar to the exponential, but also very different. There is up/down alternative divergence on one side and convergence on the other, but this time, the diverging up/diverging down pattern happens for positive $x$ (it was negative $x$ for the exponential), and the convergence happens on the left, as $x$ approaches 0. You can also see that for $x>2$ you never seem to gain any improvement if you add more terms. This is because the Taylor series for the logarithm only 'works' for a given interval, in this case for values of $x$ between 0 and 2!

This is not really a problem, however. We haven't yet seen logarithms, but one of the properties of that function is that:

$$\ln x = -\ln \frac{1}{n}$$

So suppose I wanted to use the Taylor polynomial to get an approximation of $\ln 3$. Clearly, I can't directly use the approximation for $x=3$, as we've just said it doesn't work there. I can, however, find the approximate value for $x=\frac{1}{3}$ and just flip the sign! 

In [5]:
taylor_widget(xMin_init = -2, xMax_init = 6, yMin_init = -4, yMax_init = 4, func_init = 'Logarithm', numTerms_init = 18)

For the left side of the logarithm (between $x=0$ and $x=1$), every extra term in the polynomial brings us closer to the true value, similar to what we saw for the exponential. The convergence does slow down very quickly, but it is there nonetheless.

In [6]:
taylor_widget(xMin_init = -1, xMax_init = 1, yMin_init = -6, yMax_init = 0.5, func_init = 'Logarithm', numTerms_init = 24)

#### Wrapping up: estimating unknown functions

The point of these diagrams is to show you that the reason why you can often use polynomials to approximate functions is that a lot of functions have this underlying property (being *analytical*) which means that they can, in theory, be written as a polynomial! This theoretical link between polynomials and functions is what makes them so useful... 

In applied setting, where you might have some $(x,y)$ data for an unknown function, for which you don't have an equation, you can still fit a polynomial to that data and get an approximation of the (unknown) function. 

In economics in particular, this happens all the time. In practice, we don't know what demand functions, supply functions, cost functions etc. look like, but they are useful tools in our models for making predictions. What we do have is (a) lots of data  and (b) statistical techniques that allow us to estimate the best parameters to fir the data. So we can estimate the parameters of polynomial functions to get approximations of what happens in real life. A lot of what you will be doing in econometrics is exactly this! The final note I need to make is to say that we rarely use polynomials above degree 3 (so $x^3$), relying mainly on linear or quadratic relationships. Hence why this module focuses so much on those.